# Imports and Setup
To begin, run the cell(s) below to install the necessary setup and imports.

In [1]:
!sudo apt install python3.8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,693 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.8-minimal amd64 3.8.15-1+bionic1 [762 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.8-minimal amd64 3.8.15-1+bionic1 [1,838 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main am

In [2]:
!python3.8 --version

Python 3.8.15


### TO REMOVE THE API KEY BEFORE PUSHING

In [3]:
!pip install wandb -qU

import wandb
wandb.login()
#a4ba8e56aac9ef90b9f0c3384d5c0ee26db765fc

     |████████████████████████████████| 1.9 MB 17.0 MB/s 
     |████████████████████████████████| 182 kB 66.8 MB/s 
     |████████████████████████████████| 166 kB 62.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 66.7 MB/s 
     |████████████████████████████████| 162 kB 71.5 MB/s 
     |████████████████████████████████| 162 kB 68.5 MB/s 
     |████████████████████████████████| 158 kB 45.7 MB/s 
     |████████████████████████████████| 157 kB 70.7 MB/s 
     |████████████████████████████████| 157 kB 65.6 MB/s 
     |████████████████████████████████| 157 kB 66.5 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 60.1 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 67.3 MB/s 
     |████████████████████████████████| 156 kB 64.5 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2022-10-31 06:56:54--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2022-10-31 06:56:54--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 80.9M 1s
    50K .......... .......... .......... .......... ..........  0%

In [5]:
%%bash
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    sqlite-3.39.3              |       h5082296_0         1.5 MB
    pip-22.2.2                 |   py38h06a4308_0         2.7 MB
    python-3.8.13              |       haa1d7c7_1        24.3 MB
    ruamel_yaml-0.15.100       |   py38h27cfd23_0         270 KB
    brotlipy-0.7.0             |py38h27cfd23_1003         349 KB
    readline-8.2               |       h5eee18b_0         457 KB
    zlib-1.2.13                |       h5eee18b_0         125 KB
    urllib3-1.26.12            |   py38h06a4308_0         178 KB
    ld_impl_linux-64-2.38      |       h1181459_1         732 KB
    pycosat-0.6.3              |   py38h7b6447c_1         107 KB
    pysocks-1.7.1              |   py38h06a4308_0   

sqlite-3.39.3        |  1.5 MB | ########## | 100% 
pip-22.2.2           |  2.7 MB | ########## | 100% 
python-3.8.13        | 24.3 MB | ########## | 100% 
ruamel_yaml-0.15.100 |  270 KB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
readline-8.2         |  457 KB | ########## | 100% 
zlib-1.2.13          |  125 KB | ########## | 100% 
urllib3-1.26.12      |  178 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
pycosat-0.6.3        |  107 KB | ########## | 100% 
pysocks-1.7.1        |   31 KB | ########## | 100% 
tqdm-4.64.1          |  122 KB | ########## | 100% 
pyopenssl-22.0.0     |   49 KB | ########## | 100% 
toolz-0.11.2         |   48 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
cryptography-38.0.1  |  1.5 MB | ########## | 100% 
wheel-0.37.1         |   31 KB | ########## | 100% 
conda-package-handli |  1.0 MB | ########## | 100% 
setuptools-65.4.0    |  1.4 MB | ########## | 100% 
ca-certifica

In [6]:
!conda update -n base -c defaults conda --yes

Solving environment: \ | / - \ | / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::six==1.11.0=py36h372c433_1
  - defaults/linux-64::chardet==3.0.4=py36h0f667ec_1
  - defaults/linux-64::asn1crypto==0.24.0=py36_0
- \ done

# All requested packages already installed.

Retrieving notices: ...working... done


In [7]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.8/site-packages"))

In [8]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.8/site-packages']

In [9]:
!conda init zsh

modified      /usr/local/condabin/conda
modified      /usr/local/bin/conda
modified      /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.8/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.zshrc

==> For changes to take effect, close and re-open your current shell. <==



In [10]:
!conda create --name env python=3.8 --yes

Solving environment: \ done

## Package Plan ##

  environment location: /usr/local/envs/env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.10.11 |       h06a4308_0         124 KB
    certifi-2022.9.24          |   py38h06a4308_0         154 KB
    ld_impl_linux-64-2.38      |       h1181459_1         654 KB
    libffi-3.3                 |       he6710b0_2          50 KB
    libgcc-ng-11.2.0           |       h1234567_1         5.3 MB
    libstdcxx-ng-11.2.0        |       h1234567_1         4.7 MB
    ncurses-6.3                |       h5eee18b_3         781 KB
    openssl-1.1.1q             |       h7f8727e_0         2.5 MB
    pip-22.2.2                 |   py38h06a4308_0         2.3 MB
    python-3.8.13              |       haa1d7c7_1        20.2 MB
    readline-8.2               |       h5eee18b_0         357 KB
 

In [11]:
!source /usr/local/bin/activate env

In [12]:
%%bash
pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
pip install timm==0.4.12
pip install pickle5
pip install scikit-learn
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 GB 814.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 55.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 19.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py)

tcmalloc: large alloc 2137669632 bytes == 0x2130000 @  0x7f6be4a281e7 0x4af051 0x4bf358 0x508b1c 0x59caf8 0x543434 0x4e4f3d 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4d9d16 0x4c787e 0x4c5c45 0x4d9dac 0x4c84ca 0x4c5c45 0x4d9dac 0x4c787e
tcmalloc: large alloc 2672091136 bytes == 0x817d4000 @  0x7f6be4a29615 0x508b1c 0x59caf8 0x543434 0x4e4f3d 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4d9d16 0x4c787e 0x4c5c45 0x4d9dac 0x4c84ca 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac
tcmalloc: large alloc 2137669632 bytes == 0x2130000 @  0x7f6be4a281e7 0x4c437d 0x5a4d76 0x4e4f3d 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4c787e 0x4c5c45 0x4d9dac 0x4e8197 0x4d038f 0x599299 0x4d0c5b 0x4cc005 0x4e8097 0x4cbad8 0x4c5c45 0x4e8224 0x4cbad8 0x4c5c45 0x4e8224 0x4c84ca 0x4f402b 0x4c7a5f 0x4f40

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/pipline/

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline


In [ ]:
!pip install ipywidgets
!pip install openpyxl
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.1/761.1 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.1/384.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.0/424.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.6/295.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 7.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 13.8 MB/s eta 0:00:00


# Initializing Constants

In [16]:
# INITIALIZING PATH VARIABLES (CHANGE HERE IF DIFFERENT DIRECTORY)

PATH_TO_MODEL_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/model/"
PATH_TO_VIDEO_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/video/"
PATH_TO_FEATURE_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/" #pre-readied extracted features to use
PATH_TO_FEATURERESULT_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_result/"
PATH_TO_FEATUREEXTRC_VASRGB_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/rgb/" # v-aishin extracted features
PATH_TO_FEATUREEXTRC_TSURGB_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/RGB_i3d_16frames_64000_SSD/" # TSU_I3D extracted features
PATH_TO_PIPLINE_DATA_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/" # ground truth
PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/pipline/train_output/PDAN/"

DIR_TSUSHT_MP4 = "toyota-smarthome-trimmed/mp4/" # smarthome trimmed (Not in use)
DIR_TSUSHUT_MP4 = "toyota-smarthome-untrimmed/RGB/Videos_mp4/" # smarthome untrimmed (for training)

FILE_CSJSON = "smarthome_CS_51.json"
FILE_CSJSON_MOD = "smarthome_CS_51_modified.json"
FILE_CVJSON = "smarthome_CV_51.json"

# Model Training (DEFAULT)
TO TEST ENVIRONMENT
- PDAN_TSU_RGB
- unmodified TSUI3D cs_json: /pipline/data/smarthome_cs_51.json
- unmodified feature files: /feature_extract/RGB_i3d_16frames_64000_SSD

## Training model

In [ ]:
import os
import ipywidgets as widgets

BATCH_SIZE = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='Batch Size:',
    disabled=False
)
EPOCH = widgets.BoundedIntText(
    value=14,
    min=1,
    max=2000,
    step=1,
    description='Epoch:',
    disabled=False
)
LR = widgets.BoundedFloatText(
    value = 0.0002,
    min = 0.0002,
    max = 1,
    description = "LR",
    disabled=False
)
modelbox = widgets.Box(children = [BATCH_SIZE, EPOCH, LR])
modelbox

Box(children=(BoundedIntText(value=1, description='Batch Size:', max=1000, min=1), BoundedIntText(value=14, de…

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py --help

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-model PDAN \
-num_channel 512 \
-lr $LR.value \
-kernelsize 3 \
-APtype map \
-epoch $EPOCH.value \
-batch_size $BATCH_SIZE.value \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB

Random_SEED!!!: 0
PDAN
batch_size: 1
cuda_avail True
RGB mode /content/drive/MyDrive/Toyota_Smarthome-main/feature_file/RGB_i3d_16frames_64000_SSD
100% 536/536 [00:05<00:00, 103.12it/s]
/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100% 536/536 [00:02<00:00, 216.52it/s]
you are processing PDAN
/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline/models.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
/content/

In [ ]:
from tqdm import tqdm
 
print("loading")
for i in tqdm(range(5)):
  print("test")

loading


100%|██████████| 5/5 [00:00<00:00, 12787.51it/s]

test
test
test
test
test


## Copying top 3 models into /model folder

In [ ]:
import os
import shutil

#Opening path to train_output for PDAN type model
model_list = os.listdir(PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER)

#Take the one with the latest epoch (likely highest accuracy)
#model_list that was return from os.listdir is already sort in asc with weight -> model
#so the last item in the list will be the model 
model_one = model_list[-1];
model_two = model_list[-3];
model_three = model_list[-5];
print(f"Top models: {model_one}, {model_two}, {model_three}")

# copying the 1st model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_one 
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_one
# copy the file
path = shutil.copyfile(src,dst)

#copying 2nd model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_two
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_two
# copy the file
path = shutil.copyfile(src,dst)

#copying 3rd model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_three
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_three
# copy the file
path = shutil.copyfile(src,dst)

# Model Training (VAISHIN)
TRAINING WITH VAISHIN EXTRACTED FEATURE FILES (Extract the files first)
- PDAN_TSU_RGB (you can specify if you want to)
- Modified TSUI3D cs_json: /pipline/data/smarthome_cs_51_modified.json
  - Only contains features that was extracted
- Modified feature files: /feature_extract/rgb

## Training model

In [ ]:
import os
import ipywidgets as widgets

BATCH_SIZE = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='Batch Size:',
    disabled=False
)
EPOCH = widgets.BoundedIntText(
    value=14,
    min=1,
    max=2000,
    step=1,
    description='Epoch:',
    disabled=False
)
LR = widgets.BoundedFloatText(
    value = 0.0002,
    min = 0.0002,
    max = 1,
    description = "LR",
    disabled=False
)
modelbox = widgets.Box(children = [BATCH_SIZE, EPOCH, LR])
modelbox

Box(children=(BoundedIntText(value=1, description='Batch Size:', max=1000, min=1), BoundedIntText(value=14, de…

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train_modified_vaishin.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-model PDAN \
-num_channel 512 \
-lr $LR.value \
-kernelsize 3 \
-APtype map \
-epoch $EPOCH.value \
-batch_size $BATCH_SIZE.value \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB

Random_SEED!!!: 0
PDAN
batch_size: 1
cuda_avail True
RGB mode /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/rgb
100% 536/536 [00:00<00:00, 2593.77it/s]
/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100% 536/536 [00:00<00:00, 7138.15it/s]
you are processing PDAN
/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline/models.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
/content/drive/.shortcut-ta

## Copying top 3 model into /model folder

In [ ]:
import os
import shutil

#Opening path to train_output for PDAN type model
model_list = os.listdir(PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER)

#Take the one with the latest epoch (likely highest accuracy)
#model_list that was return from os.listdir is already sort in asc with weight -> model
#so the last item in the list will be the model 
model_one = model_list[-1];
model_two = model_list[-3];
model_three = model_list[-5];
print(f"Top models: {model_one}, {model_two}, {model_three} to transfer over to /model folder")

# copying the 1st model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_one 
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_one
# copy the file
path = shutil.copyfile(src,dst)

#copying 2nd model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_two
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_two
# copy the file
path = shutil.copyfile(src,dst)

#copying 3rd model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_three
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_three
# copy the file
path = shutil.copyfile(src,dst)
  

Top models: model_epoch_0.0002_12, model_epoch_0.0002_11, model_epoch_0.0002_10 to transfer over to /model folder


## Renaming models

In [ ]:
import os
import ipywidgets as widgets

#Opening path to train_output for PDAN type model
model_list = os.listdir(PATH_TO_MODEL_FOLDER)
model_list.remove(".ipynb_checkpoints")

MODELS = widgets.Dropdown(
    options=model_list,
    description='Model:',
    disabled=False,
)
MODEL_NAME = widgets.Text(
    value='',
    placeholder='Enter Model name',
    description='New Name:',
    disabled=False
)

modelbox = widgets.Box(children = [MODELS, MODEL_NAME])
modelbox

Box(children=(Dropdown(description='Model:', options=('model_epoch_0.0002_12', 'model_epoch_0.0002_11', 'model…

In [ ]:
# Running this cell will confirm the name change for the previous cell

if(MODEL_NAME.value == ""):
    raise Exception ("New name of model is empty")


SELECTED_MODEL_TO_CHANGE = PATH_TO_MODEL_FOLDER + MODELS.value
NEW_NAME_FOR_MODEL = PATH_TO_MODEL_FOLDER + MODEL_NAME.value

os.rename(SELECTED_MODEL_TO_CHANGE, NEW_NAME_FOR_MODEL)

# Feature Extraction

V-AISHIN I3D Feature Extraction from video

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/

! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6 av==8.0.2

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/feature_extract
fatal: destination path 'video_features' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 47.4 MB/s eta 0:00:00


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/feature_extract


In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/video_features

/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/feature_extract/video_features


In [ ]:
"""
PROJ-3
R4.2 - As a User, I want to perform feature extraction on videos given, 
so I can train HOI ML models based on the TSU project.
"""
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

In [ ]:
import numpy as np
import os

# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))

dirList = os.listdir(PATH_TO_VIDEO_FOLDER + DIR_TSUSHUT_MP4)
videoList = []
for x in dirList:
  if ".mp4" in x:
    videoList.append(x)
videoListPaths = []
# Append all of them with their absolute path for feature extraction
for v in videoList:
  videoListPaths.append(PATH_TO_VIDEO_FOLDER + DIR_TSUSHUT_MP4 + v)
#print(videoListPaths)

# Set it to the videopaths in args
args.video_paths = videoListPaths

# args.video_paths = ['/content/drive/MyDrive/Toyota_Smarthome-main/video/sample_video_1.mp4'] #hardcoded 1 video for testing
# args.show_pred = True
# args.stack_size = 24
# args.step_size = 24
# args.extraction_fps = 25
# args.streams = 'flow'
args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)

# Load the extractor
extractor = ExtractI3D(args)

# Extract features from the videos in the video folder
for video_path in args.video_paths:
    video_name_ext = (video_path.split('/')[-1]) # name with .ext
    video_name = (video_path.split('/')[-1]).split('.')[0] # name for .npy files

    print(f'Now extracting for {video_name_ext}')
    feature_dict = extractor.extract(video_path)
    print(feature_dict.items())

    for k, v in feature_dict.items():
      print(k)
      print(v.shape)
      print(v)
      if k == "rgb":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/rgb/{video_name}', v)
        print(f'{video_name} saved to feature_extract/rgb/ folder \n')
      elif k == "flow":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/flow/{video_name}', v)
        print(f'{video_name} saved to feature_extract/flow/ folder \n')
      elif k == "fps":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/fps/{video_name}', v)
        print(f'{video_name} saved to feature_extract/fps/ folder \n')
      elif k == "timestamps_ms":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/timestamps_ms/{video_name}', v)
        print(f'{video_name} saved to feature_extract/timestamps_ms/ folder \n')
      else:
        raise NotImplementedError("Error: Unexpected item in feature_dict! \n")

## Checking dimension difference between v-aishin and tsui3d

In [ ]:
# checking dimensional difference between v-aishin and tsui3d
import numpy as np
import os

dirList = os.listdir(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER)
dirList.remove(".ipynb_checkpoints")

for item in dirList:
  print("v-aishin extracted: "+ item)
  n = np.load(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER + item)
  print(n.shape)



# Check TSU extracted features (4 features to be printed only)
dirList2 = os.listdir(PATH_TO_FEATUREEXTRC_TSURGB_FOLDER)

c = 0
for item in dirList2:
  print("TSU extracted: " + item)
  n = np.load(PATH_TO_FEATUREEXTRC_TSURGB_FOLDER + item)
  print(n.shape)
  c = c + 1
  if c == 3:
    break

v-aishin extracted: P02T01C07.npy
(278, 1, 1, 1024)
v-aishin extracted: P03T10C01.npy
(121, 1, 1, 1024)
v-aishin extracted: P03T19C05.npy
(445, 1, 1, 1024)
v-aishin extracted: P04T07C05.npy
(378, 1, 1, 1024)
v-aishin extracted: P03T19C04.npy
(445, 1, 1, 1024)
v-aishin extracted: P03T02C04.npy
(382, 1, 1, 1024)
v-aishin extracted: P04T07C04.npy
(378, 1, 1, 1024)
v-aishin extracted: P03T01C05.npy
(376, 1, 1, 1024)
v-aishin extracted: P02T01C06.npy
(277, 1, 1, 1024)
v-aishin extracted: P03T10C02.npy
(121, 1, 1, 1024)
TSU extracted: P25T08C03.npy
(374, 1, 1, 1024)
TSU extracted: P04T16C03.npy
(685, 1, 1, 1024)
TSU extracted: P11T09C01.npy
(1730, 1, 1, 1024)


## Converting v-aishin 2 dimension to 4 dimensions (as per tsui3d)

In [ ]:
import numpy as np
import os
# converting the v-aishin files into 4dim so that it can be used to be trained like in the TSUi3d
for item in dirList:
  n = np.load(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER + item)
  # expand by 1 dimension on axis 1 twice #https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html
  n = np.expand_dims(n, axis=1)
  n = np.expand_dims(n, axis=1)
  print(n.shape)
  # overwrite the files with the newly made numpy array
  np.save(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER + item, n)

(278, 1, 1, 1024)
(378, 1, 1, 1024)
(277, 1, 1, 1024)
(445, 1, 1, 1024)
(382, 1, 1, 1024)
(378, 1, 1, 1024)
(376, 1, 1, 1024)
(121, 1, 1, 1024)
(445, 1, 1, 1024)
(121, 1, 1, 1024)


## Validating features from v-aishin and tsui3d has the same dimensions

In [ ]:
# validating to ensure v-aishin extracted features have same dimensions as tsui3d
import numpy as np

# Check rgb extracted features
dirList = os.listdir(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER)
dirList.remove(".ipynb_checkpoints")
for item in dirList:
  print(item)
  print(np.load(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER + item))
  print(np.load(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER + item).ndim)


# Check TSU extracted features (4 features to be printed only)
dirList2 = os.listdir(PATH_TO_FEATUREEXTRC_TSURGB_FOLDER)
c = 0
for item in dirList2:
  print(item)
  print(np.load(PATH_TO_FEATUREEXTRC_TSURGB_FOLDER + item))
  print(np.load(PATH_TO_FEATUREEXTRC_TSURGB_FOLDER + item).ndim)
  c = c + 1
  if c == 3:
    break

P02T01C07.npy
[[[[0.08722213 0.72140712 0.05448091 ... 0.39927715 0.06084179
    0.03751946]]]


 [[[0.09569382 0.82684529 0.05770195 ... 0.51168567 0.01781808
    0.15733151]]]


 [[[0.1012579  0.86840081 0.1833203  ... 0.86592567 0.01320137
    0.07562003]]]


 ...


 [[[0.1191444  0.76930773 0.14157438 ... 0.77114367 0.03222982
    0.0768149 ]]]


 [[[0.09500311 0.83178908 0.10145854 ... 0.77908123 0.01364897
    0.03429199]]]


 [[[0.08086452 0.75392461 0.07707879 ... 0.32716626 0.06343492
    0.00229635]]]]
4
P03T10C01.npy
[[[[0.15121794 0.80580884 0.12891275 ... 0.41283181 0.33363453
    0.12201017]]]


 [[[0.16094175 0.8123225  0.09119987 ... 0.43803287 0.31442553
    0.08274806]]]


 [[[0.12285293 0.56032866 0.07895948 ... 0.28188574 0.27726373
    0.06590167]]]


 ...


 [[[0.13315418 0.52092046 0.13530385 ... 0.3964532  0.14642692
    0.08556429]]]


 [[[0.21958187 0.51701301 0.18194863 ... 0.82186705 0.22378688
    0.07549977]]]


 [[[0.2759794  0.63557088 0.08806912 ... 0.5

## Set up for the json file to have a certain number of videos ground truth depending on how many is inside of the folder (set up as new requirement)

This is required because the training will break if it has different amount of elements inside the file, compared to the number of feature files that is inside of the directory

**THIS IS FOR SELECTED V-AISHIN EXTRACTED FEATURE FILES ONLY**

**validate_train_test.py (without the train and test, only validate)**

In [ ]:
# test out the script here
# validate_train_test.py 

# read json files
''' Sample data format here
{"P11T15C01": 
  {
  "subset": "testing", 
  "duration": 22760, 
  "actions": [[27, 77, 987], [20, 987, 1092], [27, 1093, 3313], 
              [10, 3370, 3403], [19, 3482, 11749], [9, 10581, 10645], 
              [28, 11876, 11929], [7, 13356, 14227], [1, 13357, 13696], 
              [1, 13878, 14091], [25, 14755, 14800], [19, 15136, 21714], 
              [10, 15550, 15618], [9, 21679, 21732], [28, 21834, 21874], 
              [1, 22055, 22150], [1, 22228, 22303], [1, 22525, 22635]]
  }
}
'''

# duplicate the file of smarthome_cs_51.json
# delete those video files that wasn't extracted (into feature_extract) inside the json file
import os
import shutil
import json

# get current directory for cs_json file
src = PATH_TO_PIPLINE_DATA_FOLDER + FILE_CSJSON

# set directory for new cs_json file
dst = PATH_TO_PIPLINE_DATA_FOLDER + FILE_CSJSON_MOD

# copy the file
path = shutil.copyfile(src,dst)

# get list of all extracted feature file names from feature_extract
dirList = os.listdir(PATH_TO_FEATUREEXTRC_VASRGB_FOLDER)
dirList.remove(".ipynb_checkpoints")
# slicing chars 0 to 3 and last 4 to just have the file names
featList = []
for item in dirList:
  #item = item[4:]
  item = item[:-4]
  featList.append(item)

# modify the new file
## parse the new json file into dictionary to manage
f = open(dst)
data = json.load(f)
mod_data = {}
## find keys that are of the names in the featList inside of the data
for k in data:
  for name in featList:
    if k == name:
      mod_data[name] = data[k]

## overwrite this new data into the modified file
with open(dst, 'w') as f:
    json.dump(mod_data, f)


# R1 (completed)
Completed with binder that is on github check out the repo at:
https://github.com/IENORI/ICT3104 

# R2 (completed)

Data Exploration Section:
- Data folder to store input videos files
- Check out the different input videos (with UI)
- See playback of video

** currently only accepts .mp4, add your video file format if required

## R2.1

Video data folder is stored at: "/content/drive/MyDrive/Toyota_Smarthome-main/video/"

## R2.2
Checking out input videos (with UI)

In [ ]:
import os
import ipywidgets as widgets

#dirList = os.listdir(PATH_TO_VIDEO_FOLDER + DIR_TSUSHUT_MP4)
dirList = os.listdir(PATH_TO_VIDEO_FOLDER)
videoList = []
for x in dirList:
  if ".mp4" in x:
    videoList.append(x)

video = widgets.Dropdown(
    options = videoList,
    description='Video:',
    disabled=False,
)
video

Dropdown(description='Video:', options=('sample_video_2.mp4', 'sample_video_1.mp4', 'P02T01C06.mp4', 'test_oup…

## R2.3
See playback of video

In [ ]:
from IPython.display import Video

#VIDEO_SELECTED = PATH_TO_VIDEO_FOLDER + DIR_TSUSHUT_MP4 + video.value
VIDEO_SELECTED = PATH_TO_VIDEO_FOLDER + video.value
print(VIDEO_SELECTED)
Video(VIDEO_SELECTED, embed=True, width=800, height=800)

/content/drive/MyDrive/Toyota_Smarthome-main/video/sample_video_1.mp4


# R3
Inference Section:
- Load a pretrained model (with UI)
- Choose input video (with UI)
- See inference result in the form out output video with captions that indicte the current detected activity in each video frame

## R3.1 & R3.2
Load a pretrained model (with UI)

In [ ]:
import os
import ipywidgets as widgets

modelList = os.listdir(PATH_TO_MODEL_FOLDER)
modelList.remove(".ipynb_checkpoints")

model = widgets.Dropdown(
    options = modelList,
    description='Model:',
    disabled=False,
)

box = widgets.Box(children = [model, video])
box

Box(children=(Dropdown(description='Model:', options=('PDAN_TSU_RGB', 'model_epoch_0.0002_12', 'model_epoch_0.…

In [ ]:
#finalized chosen model and video directory

SELECTED_MODEL = PATH_TO_MODEL_FOLDER + model.value
SELECTED_VIDEO = PATH_TO_VIDEO_FOLDER + DIR_TSUSHUT_MP4 + video.value
print(SELECTED_MODEL)
print(SELECTED_VIDEO)

/content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB.value
/content/drive/MyDrive/Toyota_Smarthome-main/video/toyota-smarthome-untrimmed/RGB/Videos_mp4/sample_video_1.mp4


## R3.3 (WIP)
See inference result in the form out output video with captions that indicte the current detected activity in each video frame

In [ ]:
activity = ["Enter", "Walk", "Make_Coffee", "Get_water", "Make_coffee", "Use_Drawer", "Make_coffee.Pour_grains",
                "Use_telephone", "Leave", "Put_something_on_table", "Take_something_off_table", "Pour.From_kettle",
                "Stir_coffee/tea", "Drink.From_cup", "Dump_in_trash", "Make_tea", "Make_tea.Boil_water", "Use_cupboard",
                "Make_tea.Insert_tea_bag", "Read", "Take_pills", "Use_fridge", "Clean_dishes", "Clean_dishes.Put_something_in_sink",
                "Eat_snack", "Sit_down", "Watch_TV", "Use_laptop", "Get_up", "Drink.From_bottle", "Pour.From_bottle", "Drink.From_glass",
                "Lay_down", "Drink.From_can", "Write", "Breakfast", "Breakfast.Spread_jam_or_butter", "Breakfast.Cut_bread",
                "Breakfast.Eat_at_table", "Breakfast.Take_ham", "Clean_dishes.Dry_up", "Wipe_table", "Cook", "Cook.Cut", "Cook.Use_stove",
                "Cook.Stir", "Cook.Use_oven", "Clean_dishes.Clean_with_water", "Use_tablet", "Use_glasses", "Pour.From_can"]

# have to save the prob_val as a dictionary
# then use the dictionary to get the inner array stuff

# R4 (completed)

- Choose a dataset subfolder to use for training (with UI)
- Initialize a model with a network architecture configured in a separate .py file
- Specify name of this new model to be trained using the UI
- Set the batch_size & epochs (with UI)
- Run training sequence
- See visual elements to indicate progress of training in notebook
- After training is complete, trained model should be added to the list of pre-trained models that can be chosen in R3

*Assumption here is that it is using a set of feature files that was already prepared beforehand, they are to be placed into the /feature_files folder*

## R4.1
Choose a dataset subfolder to use if for training (with UI)

In [ ]:
import os
import ipywidgets as widgets

featureFolder = os.listdir(PATH_TO_FEATURE_FOLDER)
featureFolder.remove(".ipynb_checkpoints")

feature = widgets.Dropdown(
    options = featureFolder,
    description='Feature:',
    disabled=False,
)
feature

Dropdown(description='Feature:', options=('RGB_i3d_16frames_64000_SSD', 'v-aishin_i3d'), value='RGB_i3d_16fram…

In [ ]:
PATH_TO_SELECTED_FEATURE = PATH_TO_FEATURE_FOLDER + feature.value 
print(PATH_TO_SELECTED_FEATURE)

/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/v-aishin_i3d


## R4.2
Initialize a model with a network architecture configured in a separate .py file

From product owner: 
By this, we essentially mean that the 'new model' is just you running the train.py from scratch (PDAN folder empty, CS json file edited etc)
The network architecture is already done for you when you download the TSU github repo. So you only need to modify the train.py to work properly + create a test.py from the functions that are in the train.py provided.

**THIS IS FOR SELECTED PRE-READIED EXTRACTED FEATURE FILES ONLY**

**validate_train_test.py (without the train and test, only validate)**

In [ ]:
import shutil
import os


# Clearing contents inside of /pipline/train_output/PDAN
for f in os.listdir(PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER):
  os.remove(os.path.join(PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER, f))

# Setting up an edited cs_json containing only selected feature files
# duplicate the file of smarthome_cs_51.json
# delete those video files that wasn't extracted (into feature_extract) inside the json file
import os
import shutil
import json

# get current directory for cs_json file
src = PATH_TO_PIPLINE_DATA_FOLDER + FILE_CSJSON

# set directory for new cs_json file
dst = PATH_TO_PIPLINE_DATA_FOLDER + FILE_CSJSON_MOD

# copy the file
path = shutil.copyfile(src,dst)

# get list of all extracted feature file names from feature_file
dirList = os.listdir(PATH_TO_SELECTED_FEATURE)
dirList.remove(".ipynb_checkpoints")
# slicing chars 0 to 3 and last 4 to just have the file names
featList = []
for item in dirList:
  #item = item[4:]
  item = item[:-4]
  featList.append(item)

# modify the new file
## parse the new json file into dictionary to manage
f = open(dst)
data = json.load(f)
mod_data = {}
## find keys that are of the names in the featList inside of the data
for k in data:
  for name in featList:
    if k == name:
      mod_data[name] = data[k]

## overwrite this new data into the modified file
with open(dst, 'w') as f:
    json.dump(mod_data, f)


## R4.4 & R4.5 & R4.6

### R4.4
Set up batch size and epochs

### R4.5
Running the training sequence
- no model (initialized state, specify if you want to)
- Modified TSUI3D cs_json: /pipline/data/smarthome_cs_51_modified.json
  - Only contains the features that was extracted
- Modified feature files: /feature_files/PATH_TO_SELECTED_FEATURE

### R4.6 
See visual elements to indicate progress of training in notebook

In [ ]:
import os
import ipywidgets as widgets

BATCH_SIZE = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='Batch Size:',
    disabled=False
)
EPOCH = widgets.BoundedIntText(
    value=14,
    min=1,
    max=2000,
    step=1,
    description='Epoch:',
    disabled=False
)
LR = widgets.BoundedFloatText(
    value = 0.0002,
    min = 0.0002,
    max = 1,
    description = "LR",
    disabled=False
)
modelbox = widgets.Box(children = [BATCH_SIZE, EPOCH, LR])
modelbox

Box(children=(BoundedIntText(value=1, description='Batch Size:', max=1000, min=1), BoundedIntText(value=14, de…

In [ ]:
print(PATH_TO_SELECTED_FEATURE)
print(LR.value)
print(EPOCH.value)
print(BATCH_SIZE.value)

/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/v-aishin_i3d
0.0002
11
1


In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train_modified_custom.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-rgb_root $PATH_TO_SELECTED_FEATURE \
-model PDAN \
-num_channel 512 \
-lr $LR.value \
-kernelsize 3 \
-APtype map \
-epoch $EPOCH.value \
-batch_size $BATCH_SIZE.value \
-comp_info TSU_CS_RGB_PDAN 

Random_SEED!!!: 0
PDAN
batch_size: 1
cuda_avail True
RGB mode /content/drive/MyDrive/Toyota_Smarthome-main/feature_file/v-aishin_i3d
100% 10/10 [00:04<00:00,  2.01it/s]
/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100% 10/10 [00:01<00:00,  8.54it/s]
you are processing PDAN
/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline/models.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
/content/drive/.shortcut-targ

## R4.7
After training is complete, trained model should be added to the list of pre-trained models that can be chosen in R3

In [ ]:
import os
import shutil

#Opening path to train_output for PDAN type model
model_list = os.listdir(PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER)

#Take the one with the latest epoch (likely highest accuracy)
#model_list that was return from os.listdir is already sort in asc with weight -> model
#so the last item in the list will be the model 
model_one = model_list[-1];
model_two = model_list[-3];
model_three = model_list[-5];
print(f"Top models: {model_one}, {model_two}, {model_three} to transfer over to /model folder")

# copying the 1st model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_one 
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_one
# copy the file
path = shutil.copyfile(src,dst)

#copying 2nd model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_two
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_two
# copy the file
path = shutil.copyfile(src,dst)

#copying 3rd model file into /model folder
src = PATH_TO_PIPLINE_TRAINOUTPUT_PDAN_FOLDER + model_three
# set directory for the /model folder
dst = PATH_TO_MODEL_FOLDER + model_three
# copy the file
path = shutil.copyfile(src,dst)

Top models: model_epoch_0.0002_10, model_epoch_0.0002_8, model_epoch_0.0002_1 to transfer over to /model folder


## R4.3

Specify name of this new model to be trained using the UI

In [ ]:
import os
import ipywidgets as widgets

#Opening path to train_output for PDAN type model
model_list = os.listdir(PATH_TO_MODEL_FOLDER)
model_list.remove(".ipynb_checkpoints")

MODELS = widgets.Dropdown(
    options=model_list,
    description='Model:',
    disabled=False,
)
MODEL_NAME = widgets.Text(
    value='',
    placeholder='Enter Model name',
    description='New Name:',
    disabled=False
)

modelbox = widgets.Box(children = [MODELS, MODEL_NAME])
modelbox

Box(children=(Dropdown(description='Model:', options=('model_epoch_0.0002_11', 'model_epoch_0.0002_10', 'lates…

In [ ]:
# Running this cell will confirm the name change for the previous cell

if(MODEL_NAME.value == ""):
    raise Exception ("New name of model is empty")


SELECTED_MODEL_TO_CHANGE = PATH_TO_MODEL_FOLDER + MODELS.value
NEW_NAME_FOR_MODEL = PATH_TO_MODEL_FOLDER + MODEL_NAME.value

os.rename(SELECTED_MODEL_TO_CHANGE, NEW_NAME_FOR_MODEL)

# R5 (completed)

## R5.1
Choose a dataset subfolder, using appropriate UI elements, from the data folder on the TSU project

In [17]:
import os
import ipywidgets as widgets

featureFolder = os.listdir(PATH_TO_FEATURE_FOLDER)
featureFolder.remove(".ipynb_checkpoints")

feature = widgets.Dropdown(
    options = featureFolder,
    description='Features:',
    disabled=False,
)
feature

Dropdown(description='Features:', options=('RGB_i3d_16frames_64000_SSD', 'v-aishin_i3d'), value='RGB_i3d_16fra…

In [ ]:
PATH_TO_SELECTED_FEATURE = PATH_TO_FEATURE_FOLDER + feature.value 
print(PATH_TO_SELECTED_FEATURE)

/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/v-aishin_i3d


## R5.2
Load a pretrained model using an appropriate UI component

In [18]:
import os
import ipywidgets as widgets

modelList = os.listdir(PATH_TO_MODEL_FOLDER)
modelList.remove(".ipynb_checkpoints")

MODEL = widgets.Dropdown(
    options = modelList,
    description='Model:',
    disabled=False,
)
BATCH_SIZE = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='batch_size:',
    disabled=False
)
EPOCH = widgets.BoundedIntText(
    value=1,
    min=1,
    max=2000,
    step=1,
    description='epoch:',
    disabled=False
)
LR = widgets.BoundedFloatText(
    value = 0.0002,
    min = 0.0002,
    max = 1,
    description = "LR",
    disabled=False
)

box = widgets.Box(children = [MODEL, BATCH_SIZE, EPOCH, LR])
box

Box(children=(Dropdown(description='Model:', options=('model_epoch_0.0002_11', 'model_epoch_0.0002_10', 'lates…

## R5.3 & R5.4 & R5.5 & R5.6

### R5.3
Run the testing sequence i.e. perform inference on each data sample and accumulate some statistics

**The statistics for each video is saved to: /feature_result folder**

### R5.4
See some visual elements to indicate the progress of testing in the notebook

### R5.5
View some result (Average Precision per activity class and mean Average Precision) to know how well model performed

### R5.6
Save the results to the results folder of the repo:
Results are saved at: **/feature_result folder**

**Further information on the training of the model, refer to the website: https://wandb.ai/3104_ml**

There are also wandb logs when you train the model, you can also opt to access it through that method instead


In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/test_modified_custom.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-rgb_root $PATH_TO_SELECTED_FEATURE \
-model PDAN \
-num_channel 512 \
-lr $LR.value \
-kernelsize 3 \
-APtype map \
-epoch $EPOCH.value \
-batch_size $BATCH_SIZE.value \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/$MODEL.value

Random_SEED!!!: 0
PDAN
batch_size: 1
cuda_avail True
RGB mode /content/drive/MyDrive/Toyota_Smarthome-main/feature_file/v-aishin_i3d
100% 10/10 [00:00<00:00, 128.89it/s]
/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100% 10/10 [00:00<00:00, 734.59it/s]
you are processing PDAN
/content/drive/.shortcut-targets-by-id/1LWR6jlRpEpJnHZO4yBjJ5EPNy1QfYCYq/Toyota_Smarthome-main/pipline/models.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
/content/drive/.shortcut-ta

# R6 
Configure notebook to use appropriate UI elements coupled with the right .py modules so taht R2-5 can be performed based on another pipeline

# R10 (new).......

New requirement :
1) Implement MS-TCT into the notebook as well. All instructions to set up etc can be found in the repo : https://github.com/dairui01/MS-TCT
Reminder to all: Since everyone’s 2nd algorithm is STEP - ensure that when you’re using the AVA dataset (linked in the STEP algo) , you’re using the v2.2 . Also ensure that I can easily bring in my own videos as is written on the GitHub page of STEP , with markdown text details on how we can set up our own videos. You can try this out with 2-3 TSU videos and showing the results (should not take much time as the pipeline should be easy to use after you’ve set it up)

# R7 (completed)
- A thorough description of how to interact with the cells in each section
- Technical implementation details of the underlying implementation for each section
- URL references with descriptions of how they are used in the implementation


# R8 (completed)
All requirements here should be easily traceable

# R9 (completed)
Adhere to secure coding practices